In [1]:
import os
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline


import glob
from astropy.io import fits
from astropy.stats import sigma_clip
from scipy.ndimage import interpolation as interp

#from skimage.feature.register_translation import (register_translation, _upsampled_dft)

# import aotools
import astroscrappy
from scipy.optimize import leastsq
# import ccdproc
import math

In [2]:
from scipy.ndimage import median_filter, gaussian_filter


def guess_gaussian_parameters(d):
    """
    This image guesses the maximum intensity of the 
    image by obtaining a smoothed version of the image 
    via median + gaussian filtering. Then, finds the 
    maximum of the smoothed image. Using the median-filtered 
    image, the algorithm also estimates the width of the PSF and 
    with this value and an estimate of the volume, the amplitude of 
    such gaussian.

    Input

    d       Numpy array that contains the values of the pixels.

    Output

    x0      x coordinate of the image maximum
    
    y0      y coordinate of the image maximum

    sigma   Width of the PSF

    A       Estimated amplitude of the gaussian function
    """

    # First, smooth the image with a median filter. For this, find 
    # the optimal window as the square-root of the geometric mean 
    # between the sizes of the array. This step is good to kill outliers:
    window = int(np.sqrt(np.sqrt(d.shape[0] * d.shape[1])))
    if window % 2 == 0:
        window = window + 1

        # Originally there was a median filter here, but it really adds a lot of time to the routine
    #d_mfiltered = median_filter(d,size = window)
    # Going to try without it to see if it makes a significant difference for vetoing
    d_gfiltered = gaussian_filter(d, sigma=window)

    # Now, find the maximum of this image:
    y0, x0 = np.where(d_gfiltered == np.max(d_gfiltered))

    # Take the first element. This helps in two cases: (1) only one maximum has 
    # been found, the outputs are numpy arrays and you want to extract the numbers 
    # and (2) in case there are several maximums (this has not happened so 
    # far but could in principle), pick the first of them:
    y0 = y0[0]
    x0 = x0[0]

    # Now estimate the width of the PSF by taking a "cross-plot" using the 
    # maximum values found:
    x_cut = d[:, int(x0)]
    sigma_x = (np.sum(x_cut * (np.abs(np.arange(len(x_cut)) - y0))) / np.sum(x_cut)) / 3.
    y_cut = d[int(y0), :]
    sigma_y = (np.sum(y_cut * (np.abs(np.arange(len(y_cut)) - x0))) / np.sum(y_cut)) / 3.
    sigma = np.sqrt(sigma_x * sigma_y)

    # (Under) estimate amplitude assuming a gaussian function:
    A = (np.sum(d - np.median(d)) / (2. * np.pi * sigma ** 2))

    return x0, y0, sigma, 2. * A


In [3]:
datadir = '/home/tehan/PycharmProjects/Shane-AO-Reduction/Raw_Data/data-2024-01-28-AO-Paul.Robertson/'
os.chdir(datadir)

#These are the id numbers of the fitz files
firstID = 1;
lastID = 9999;

##Let's go through our images and remove the bad frames

all_object_list = []

veto_list = []

#Let's just make a list of every object
for ID in range(firstID, lastID + 1):
    try:
        if ID < 100:
            if ('s00' + str(ID) + '.fits'):
                header = fits.getheader('s00' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s00' + str(ID) + '.fits')


        elif 99 < ID < 1000:
            if ('s0' + str(ID) + '.fits'):
                header = fits.getheader('s0' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s0' + str(ID) + '.fits')

        elif 999 < ID < 10000:
            if ('s' + str(ID) + '.fits'):
                header = fits.getheader('s' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s' + str(ID) + '.fits')

    except OSError:
        continue
    except KeyError:
        continue

flat_obj_list = []
#let's remove the darks
for object in all_object_list:
    header = fits.getheader(object)
    obj = header['OBJECT']
    if obj == 'dark':
        continue
    else:
        flat_obj_list.append(object)




In [4]:
#Now we go through each object here, and we make a list of all the bad ones

veto_list = []

for obj in flat_obj_list:
    header = fits.getheader(obj)
    try:
        image_data = fits.getdata(obj)
        center=(1100,730)
        image_window = image_data[(center[1] - 200):(center[1] + 200), (center[0] - 200):(center[0] + 200)]  # we use the window we care about
        # shrink the window to avoid companion

        print(obj)
        print(header['OBJECT'])
        if 'flat' in header['OBJECT'].lower():
            cent_counts = np.mean(image_window)
            print('Flat!')
        else:

            x0, y0, sigma, A = guess_gaussian_parameters(
                image_window)  #need to do this for the window, not the whole image

            print(image_window[y0,x0])
            #now that we have the centroid, let's make a window about the centroid, and take the average
            centroid_window = image_window[y0 - 1:y0 + 1, x0 - 1:x0 + 1]
            cent_counts = np.mean(centroid_window)  #let's try max instead of mean

        print(cent_counts)

        if cent_counts > 25000:  #Setting our max counts at 25000 here
            veto_list.append(obj)
            print('Appended!')
        elif cent_counts < 500:  #Setting our minimum counts at 6500 here
            veto_list.append(obj)
            print('Appended!')
        elif math.isnan(cent_counts) == True:
            veto_list.append(obj)
            print('Appended!')
        else:
            continue
    except ValueError:
        continue


#Just median filter: ~94 s, appears accurate
#Just Gaussian filter: ~ 1 s, appears accurate



In [5]:
#Write it to a text file for later
with open('veto_list.txt', 'w') as f:
    for item in veto_list:
        f.write("%s\n" % item)